# Longformer
* arXiv [paper](https://arxiv.org/pdf/2004.05150.pdf)
* Huggingface [documentation](https://huggingface.co/docs/transformers/model_doc/longformer)
* Medium [blog](https://medium.com/dair-ai/longformer-what-bert-should-have-been-78f4cd595be9#:~:text=The%2030%20layer%20model%2C%20when,(102M%20vs%20277M%20parameters)). Check graphics of memory?


config.attention_window
global_attention_mask
* 0: the token attends “locally”,
* 1: the token attends “globally”.


Problem of Longformer: Since it's based on window. It might miss elements far from the current token. This might be crucial in tables, for example, where the column name might be many tokens (sequencially ordered) before. Hence, loosing the ability to attend freely to any other token in the document, might 'suponer' loss of information and decrease of performance.
Also, it allows up to 4,096 tokens, where we should allow much more...

Longformer: 
 * Everything global: Full context
 * Only questions global: Window of w

Since it's BERT-like it is an extractive question ansering method.
(Comes from RoBERTa)

In [ ]:
# Code in forward:  https://github.com/huggingface/transformers/blob/v4.18.0/src/transformers/models/longformer/modeling_longformer.py#L1964
if global_attention_mask is None:
    global_attention_mask = _compute_global_attention_mask(input_ids, self.config.sep_token_id)


Up to 4,096 tokens?
config.attention_window.

In [ ]:
# Code in forward:  https://github.com/huggingface/transformers/blob/v4.18.0/src/transformers/models/longformer/modeling_longformer.py#L1964
if global_attention_mask is None:
    global_attention_mask = _compute_global_attention_mask(input_ids, self.config.sep_token_id)


Up to 4,096 tokens?
config.attention_window.

In [ ]:
len(model.config.attention_window)  # len(attention_window) == num_hidden_layers - https://huggingface.co/docs/transformers/model_doc/longformer#transformers.LongformerConfig.attention_window

In [ ]:
Global attention mask: https://huggingface.co/docs/transformers/v4.18.0/en/model_doc/longformer#transformers.LongformerModel.forward.global_attention_mask

# First simple test

In [ ]:
import torch
from transformers import LongformerTokenizer, LongformerForQuestionAnswering

In [2]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")
model = LongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")

In [3]:
# question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
question, text = ["Who was Jimmy Hendrinx?", "Who was Jim Henson?"], ["Jim Henson was a nice puppet", "Jim Henson was a nice dog"]
encoding = tokenizer(question, text, return_tensors="pt", padding=True)
input_ids = encoding["input_ids"]

In [4]:
# default is local attention everywhere
# the forward method will automatically set global attention on question tokens
attention_mask = encoding["attention_mask"]

outputs = model(input_ids, attention_mask=attention_mask)

In [5]:
start_logits = outputs.start_logits
end_logits = outputs.end_logits

start_idxs = torch.argmax(start_logits, axis=1)
end_idxs = torch.argmax(end_logits, axis=1)

for bs in range(len(input_ids)):
    all_tokens = tokenizer.convert_ids_to_tokens(input_ids[bs].tolist())

    answer_tokens = all_tokens[start_idxs[bs] : end_idxs[bs] + 1]
    answer = tokenizer.decode(
        tokenizer.convert_tokens_to_ids(answer_tokens)
    )  

    answer = answer.strip() # remove space prepending space token
    print(answer)


puppet
dog


**Com tindria sentit que això funcionés:**

Que el LongformerForQuestionAnswering fes herència del forward del Longformer on acceptés dos inputs, un per la pregunta i un altre pel context. A la pregunta automàticament posaria global attention = 1 i al context global attention = 2.

No obstant a la [línia 1980](https://github.com/huggingface/transformers/blob/v4.18.0/src/transformers/models/longformer/modeling_longformer.py#L1964) no sembla que hi hagi res d'això... Penso que potser el *\_\_call()\_\_* no crida al *forward()* directament sinó que fa un preprocessament... hauria de posar el codi a Pytorch i debuggar per veure exactament quin és el flow del procés.

# Baseline DocVQA

In [1]:
import os
import numpy as np

import torch
from transformers import LongformerTokenizer, LongformerForQuestionAnswering

In [2]:
root_dir = "/SSD/Datasets/DocVQA/Task1/pythia_data"
singledocvqa_dir = os.path.join(root_dir, "imdb/docvqa/")

In [11]:
train_imdb = np.load(os.path.join(singledocvqa_dir, 'new_imdb_train.npy'), allow_pickle=True)
val_imdb = np.load(os.path.join(singledocvqa_dir, 'new_imdb_val.npy'), allow_pickle=True)
test_imdb = np.load(os.path.join(singledocvqa_dir, 'new_imdb_test.npy'), allow_pickle=True)

In [6]:
train_imdb[0]

{'creation_time': 1637157799.630989,
 'version': 1.1,
 'dataset_type': 'train',
 'has_answer': True}

In [12]:
train_imdb=train_imdb[1:]
val_imdb = val_imdb[1:]
test_imdb = test_imdb[1:]

In [10]:
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")
model = LongformerForQuestionAnswering.from_pretrained("allenai/longformer-large-4096-finetuned-triviaqa")

In [ ]:
# question, text = "Who was Jim Henson?", "Jim Henson was a nice puppet"
question, text = ["Who was Jimmy Hendrinx?", "Who was Jim Henson?"], ["Jim Henson was a nice puppet", "Jim Henson was a nice dog"]
encoding = tokenizer(question, text, return_tensors="pt", padding=True)
input_ids = encoding["input_ids"]

In [21]:
for qa in val_imdb:
    question = qa['question']
    context = ' '.join([word.lower() for word in qa['ocr_tokens']])
    encoding = tokenizer(question, context, return_tensors="pt", padding=True)
    input_ids = encoding["input_ids"]
    break

In [23]:
encoding['attention_mask']

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])

In [24]:
question

'What is the ‘actual’ value per 1000, during the year 1975?'